## Set Up



In this notebook, I will be exploring the three approaches of preesenting SQL in a Jupyter Notebook demonstrated in [lecture 20 from DATA 100](https://ds100.org/fa24/resources/assets/lectures/lec20/lec20.html). The three approaches are SQL Magic, `pandas`, and DuckDB.

In [1]:
#pip install jupysql --upgrade

In [2]:
import sqlite3

In [3]:
%load_ext sql

In [4]:
#%pip install duckdb-engine --quiet

In [ ]:
import duckdb

# Create a DuckDB database file (example.duckdb)
conn = duckdb.connect('example_duck.db')

# Optional: Create a table
conn.execute('''
    CREATE TABLE IF NOT EXISTS Dragon (
        name TEXT PRIMARY KEY,
        year INTEGER,
        cute INTEGER
    )
''')

# Optional: Insert data
conn.execute('''
    INSERT INTO Dragon (name, year, cute) VALUES
    ('hiccup', 2010, 10),
    ('drogon', 2011, -100),
    ('dragon 2', 2019, 0),
    ('puff', 2010, 100),
    ('smaug', 2011, NULL)
''')

# Save changes and close the connection
conn.close()


Need to connect to duckdb.

In [9]:
%sql duckdb:///example_duck.db

Connecting to 'duckdb:///example_duck.db'

In [10]:
%%sql 
SELECT * 
FROM Dragon;

Running query in 'duckdb:///example_duck.db'

name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None
